In [ ]:
# pip install requests
# pip install -U ipykernel
# pip install tabulate

In [ ]:
import requests
URL = 'https://online-price-watch.consumer.org.hk/opw/opendata/pricewatch.json'
response = requests.get(URL)
result = response.json()
print('Count: %s' % (len(result)))

In [ ]:
import math

list = [
    item for item in result
    if len(item['prices']) > 1
]
print('Count: %s' % (len(list)))

for item in list:
    maxValue = 0
    minValue = math.inf
    minValue2 = math.inf
    highPrice = item['prices'][0]
    lowPrice = highPrice
    lowPrice2 = None
    for price in item['prices']:
        if price['supermarketCode'] == 'ZTORE':
            continue
        value = float(price['price'])
        if value > maxValue:
            maxValue = value
            highPrice = price
        if value < minValue:
            minValue = value
            lowPrice = price
        if minValue <= value and value < minValue2 and value < maxValue and price is not lowPrice:
            minValue2 = value
            lowPrice2 = price
    
    item['high_price'] = highPrice
    item['low_price'] = lowPrice
    item['low_price2'] = lowPrice2
    item['price_diff'] = maxValue - minValue
    item['price_diff_percent'] = math.floor(minValue / maxValue * 100)
    
print(list[0])

In [ ]:
from tabulate import tabulate

table = [
    ["Sun",696000,1989100000],
    ["Earth",6371,5973.6],
    ["Moon",1737,73.5],
    ["Mars",3390,641.85]
]
print(tabulate(table))

In [ ]:
def get_price_diff(item):
    return item.get('price_diff')

def get_price_diff_percent(item):
    return item.get('price_diff_percent')

list.sort(key=get_price_diff_percent)

table = [
    [
        item['code'], 
        # item['cat1Name']['zh-Hant'],
        '%s - %s' % (item['brand']['zh-Hant'], item['name']['zh-Hant']),
        # item['price_diff'], 
        item['price_diff_percent'],
        # item['low_price']['supermarketCode'], item['low_price']['price'], 
        # item['high_price']['supermarketCode'], item['high_price']['price'], 
        ('%s (%s), %s (%s)' % (item['low_price']['price'], item['low_price']['supermarketCode'], item['high_price']['price'],  item['high_price']['supermarketCode'])) if item['low_price2'] is None else ('%s (%s), %s (%s), %s (%s)' % (item['low_price']['price'], item['low_price']['supermarketCode'], item['low_price2']['price'], item['low_price2']['supermarketCode'], item['high_price']['price'],  item['high_price']['supermarketCode']))
    ]
    for item in list
    if item['price_diff'] > 0
]

print(tabulate(
    table, 
    headers=[
        'Code', 'Name', 
        # 'Highest - Lowest', 
        'Lowest / Highest (%)',
        # 'Supermarket (Lowest Price)', 'Lowest Price',
        # 'Supermarket (Highest Price)', 'Highest Price',
        'Price [Lowest, 2nd Lowest, Highest]',
    ],
    tablefmt="simple",
    floatfmt=".1f",
    numalign="left"
))